## Import dos dados

In [1]:
import string
from pathlib import Path
from collections import Counter
import math
import re
import pandas as pd
import numpy as np

In [2]:
datasets_dir = Path().absolute().parent.joinpath('datasets')
reviews = f"{datasets_dir}/imdb-reviews-pt-br.csv"

In [3]:
df = pd.read_csv(reviews)
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [4]:
df.groupby('sentiment').count()

,id,text_en,text_pt
sentiment,,,
neg,24765,24765,24765
pos,24694,24694,24694


In [5]:
neg_sentences = 24765
pos_sentences = 24694

## Tratamento do texto e treinamento

In [6]:
stopwords = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para', 'é', 'com', 'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 
             'há', 'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'estão', 'você', 'tinha', 'foram', 'essa', 'num', 
             'nem', 'suas', 'meu', 'às', 'minha', 'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas' 
             'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 
             'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos', 'estiverem', 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 
             'houveram', 'houvera', 'houvéramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvéssemos', 'houvessem', 'houver', 'houvermos', 'houverem', 'houverei', 'houverá', 'houveremos', 'houverão', 'houveria', 'houveríamos', 'houveriam', 'sou', 
             'somos', 'são', 'era', 'éramos', 'eram', 'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'seja', 'sejamos', 'sejam', 'fosse', 'fôssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 'seríamos', 
             'seriam', 'tenho', 'tem', 'temos', 'tém', 'tinha', 'tínhamos', 'tinham', 'tive', 'teve', 'tivemos', 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 'tivéssemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 
             'terei', 'terá', 'teremos', 'terão', 'teria', 'teríamos', 'teriam', '.', ',']

positive_rev = []
negative_rev = []

In [7]:
del df['id']
del df['text_en']

In [8]:
for index, row in df.iterrows():
    if(row['sentiment'] == 'neg'):
        negative_rev.append(row['text_pt'])
    elif (row['sentiment'] == 'pos'):
        positive_rev.append(row['text_pt'])

In [9]:
#Separando para testes
training_negative_rev = negative_rev[18573:]
training_positive_rev = positive_rev[18520:]
testing_negative_rev = negative_rev[:18574]
testing_positive_rev = positive_rev[:18520]
testing_reviews = testing_negative_rev + testing_positive_rev

In [10]:
def remove_stopword(dictionary):
    delete = [key for key in dictionary if key in stopwords]
    for key in delete: del dictionary[key]
    return dictionary     

In [11]:
def format_text(array):
    normalized = [str.lower(sentence) for sentence in array]
    split = [re.findall(r"[\w']+|[,.!?;]", item) for item in normalized]
    flat_split = []
    for lists in split:
        for words in lists:
            flat_split.append(words)
    return flat_split

In [12]:
def format_text_count(array):
    normalized = [str.lower(sentence) for sentence in array]
    split = [re.findall(r"[\w']+|[,.!?;]", item) for item in normalized]
    counter = [Counter(word) for word in split]
    summed = Counter({})
    for word in counter:
            summed += Counter(word)
    return summed   

In [13]:
negatives_word_count = format_text_count(training_negative_rev)
positives_word_count = format_text_count(training_positive_rev)

In [14]:
negative_words = remove_stopword(negatives_word_count)
positive_words = remove_stopword(positives_word_count)

In [15]:
def cal_number_of(dictionary):
    numbers = 0
    for value in dictionary.values():
       numbers += value
    return numbers    

In [16]:
numb_of_negative_words = cal_number_of(negative_words)
numb_of_positive_words = cal_number_of(positive_words)
numb_of_words = numb_of_negative_words + numb_of_positive_words

## Classificação

In [17]:
#log P(C)
prob_of_neg = math.log((neg_sentences) / (pos_sentences+neg_sentences), 10)
prob_of_pos = math.log((pos_sentences) / (pos_sentences+neg_sentences), 10)

In [18]:
#E log(P(Wi|c))
def calc_prob_of_word_in_context(word, context):
    if(context == 'neg'):
        probability = math.log((negative_words[word] + 1) / (numb_of_negative_words + numb_of_words), 10)
    elif(context =='pos'):
        probability = math.log((positive_words[word] + 1) / (numb_of_positive_words + numb_of_words), 10)
    return probability    

In [19]:
def ascertain_context(review):
    prob_of_negative = 0.0
    prob_of_positive = 0.0
    prob_of_words_in_negative = 0.0
    prob_of_words_in_positive = 0.0
    for words in review:
        prob_of_words_in_negative += (calc_prob_of_word_in_context(words, 'neg'))
        prob_of_words_in_positive += (calc_prob_of_word_in_context(words, 'pos'))
    
    prob_of_negative = prob_of_neg + prob_of_words_in_negative
    prob_of_positive = prob_of_pos + prob_of_words_in_positive
    
    if(prob_of_negative > prob_of_positive):
        return 'Negative'
    elif(prob_of_negative < prob_of_positive):
        return 'Positive'

In [21]:
class Classification():
    selected_number = input("Selecione um número de 0 a 37093: ")
    print("Review escolhida: ", testing_reviews[int(selected_number)])
    split_selected_text = format_text([testing_reviews[int(selected_number)]])
    print(ascertain_context(split_selected_text))

Selecione um número de 0 a 37093:  37090


Review escolhida:  Eu não vi esse filme há anos, então meu conhecimento é um pouco enferrujado. Eu me lembro de pensar que este filme é o dobro do filme de Braveheart. É simplesmente mais realista e tem personagens mais críveis. Ridb Roy parece que se pode imaginar que Rob Roy se parece com um cabelo bagunçado e barba com roupas simples. Também o Liam Nesson tem um olhar celta sobre ele, ele parece um escocês e, mais importante, se parece com Rob Roy. É uma comparação que irrita algumas pessoas, mas o compara a Mel Gibson como William Wallace. Gibson supostamente está interpretando um homem cuja lenda fez com que ele fosse descrito como um gigante de 6 pés 7, enquanto Gibson é quase um pé mais baixo. A história contém um pouco de romance, conspiração e uma história de azarão. Soa qualquer coisa como Braveheart? Mas em vez de um filme que grita "liberdade", liberdade e nacionalismo, temos um filme que diz respeito a honra, amor e justiça. Isso torna um filme mais interessante. Muito par

In [ ]:
# negative examples: 1 and 8
#positive examples: 37090  and 37093